<a href="https://colab.research.google.com/github/JaiswalFelipe/deep-segmentation/blob/master/mydataloader.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import os
import numpy as np
import imageio
import torch

from PIL import Image
from torch.utils.data import Dataset
from skimage import transform
from skimage import img_as_float
from matplotlib import pyplot as plt
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from torchvision import transforms

from data_utils import create_or_load_statistics, create_distrib, normalize_images, data_augmentation

scaler = MinMaxScaler()

In [6]:
class NGDataset(Dataset):
  def __init__(self, img_dir, mask_dir, img_list, mean, std, transform=None, if_test = False):

    self.img_dir = img_dir
    self.mask_dir = mask_dir
    self.img_list = img_list
    self.mean = mean
    self.std = std
    self.transform = transform
    self.if_test = if_test
    #self.images = os.listdir(img_dir)

  def __len__(self):
    return len(self.img_list)

  def __getitem__(self, index):
    
    image = img_as_float(imageio.imread(os.path.join(self.img_dir, self.img_list[index] + '.tif')))
    mask = imageio.imread(os.path.join(self.mask_dir, self.img_list[index] + '.tif')).astype(int)

    if self.transform is not None:
      transformed_img = self.transform(image = image, mask = mask)

      image = transformed_img['image']
      mask = transformed_img['mask']

    #transformed_images = transform.Compose([transform.ToTensor(), transform.Normalize(self.mean, self.std)])

    # Normalization.
    normalize_images(image, self.mean, self.std) # check data_utils.py
    
    if self.if_test:
      pass 
    else: 
      image, mask = data_augmentation(image, mask)
     
    image = np.transpose(image, (2, 0, 1))

    # Turning to tensors.
    image = torch.from_numpy(image.copy())
    mask = torch.from_numpy(mask.copy())

    # Returning to iterator.
    return image.float(), mask


In [11]:
img_path = '/content/drive/MyDrive/training_patches/image_dataset'
mask_path = '/content/drive/MyDrive/training_patches/mask_dataset'

img_list = []
for (_,_,files) in os.walk(img_path):
     for file in files:   
        img_list.append(file.split('.')[0])
        #print (files)
        #print ('--------------------------------')

In [8]:
train_loader = NGDataset(img_path, mask_path, img_list, )